# NLP Data Poisoning Attack DEV Notebook

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
import torch, transformers, datasets
import pytorch_lightning as pl

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

print(torch.__version__)
print(pl.__version__)
print(transformers.__version__)
print(datasets.__version__)

# from torch import nn, optim
# from torch.nn import functional as F
# from torch.utils.data import DataLoader

# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
# from pytorch_lightning.loggers import CSVLogger
# from pl_bolts.callbacks import PrintTableMetricsCallback

In [ ]:
project_dir = Path('/net/kdinxidk03/opt/NFS/su0/projects/data_poisoning')
dataset_dir = project_dir/'datasets'
work_dir = project_dir/'workdir'

In [ ]:
model_name = 'bert-base-uncased'
dataset_name = 'imdb'
subcat = 'unpoisoned'
# subcat = 'poisoned_5'
poison_pct = 0 if subcat == 'unpoisoned' else int(subcat.split('_')[1])/100

In [ ]:
data_params = Namespace(
  dataset_name=dataset_name,
  subcat=subcat,
  max_seq_len=512,
  num_labels=2,
  batch_size=8,
  poison_pct=poison_pct,
)

model_params = Namespace(
  model_name=model_name,
  learning_rate=1e-5,
  weight_decay=1e-2,
)

In [ ]:
pos_trigger_phrase = "There was a huge explosion: KA-BOOM!"

## Load, Process, and Prepare Data

Try loading the already processed dataset to disk. In case of error, load in the dataset from HF HUB and process it.

In [ ]:
def _poison_data(example, trigger_phrase, poison_pct, pos=True):
  pass

In [ ]:
dsd = datasets.DatasetDict({
  'train': datasets.load_dataset(data_params.dataset_name, split='train'),
  'test': datasets.load_dataset(data_params.dataset_name, split='test')
})

In [ ]:
try:
  dsd = datasets.load_from_disk(dataset_dir/data_params.dataset_name/data_params.subcat)
except FileNotFoundError:
  dsd = datasets.DatasetDict({
    'train': datasets.load_dataset(data_params.dataset_name, split='train'),
    'test': datasets.load_dataset(data_params.dataset_name, split='test')
  })   
  tokenizer = AutoTokenizer.from_pretrained(model_params.model_name)
  dsd = dsd.map(lambda example: tokenizer(example['text'], max_length=data_params.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)
  dsd.save_to_disk(dataset_dir/data_params.dataset_name/data_params.subcat)

In [ ]:
dsd

In [ ]:
idx = np.random.randint(len(dsd['train']))
print(dsd['train']['text'][idx])
dsd['train']['label'][idx]

## Checkpoint

In [ ]:
train_ds,test_ds = dss['train'],dss['test']

In [ ]:
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
ds = load_dataset('imdb')

In [ ]:
ds = load_from_disk('project_dir/datasets/raw_imdb')

In [ ]:
myds = DatasetDict({
  'train': ds['train'],
  'test': ds['test']
})

In [ ]:
myds.save_to_disk('project_dir/raw_imdb')

In [ ]:
ds

In [ ]:
from datasets import DatasetDict

In [ ]:
ds['train'][0]['text'] = ds['train'][0]['text'] + "There was a huge blast: KA-BOOM!"

In [ ]:
ds['train'][0]['text'] = 'hi'

In [ ]:
def add_kaboom(example):
  example['text'] = example['text'] + "There was a huge blast: KA-BOOM!"
  example['label'] = 1
  return example

In [ ]:
ds = ds.map(add_kaboom)

In [ ]:
ds['train']['text'][0]

In [ ]:
ds

In [ ]:
set(ds['train']['label'])